In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

# WithOut Pipeline

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [8]:
x = data.drop(['Survived','PassengerId','Name','Ticket','Cabin'],axis=1)

In [9]:
y = data['Survived']

In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [11]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

In [12]:
x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

In [13]:
x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])

In [14]:
ohe_sex = OneHotEncoder(drop='first',sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(drop='first',sparse=False, handle_unknown='ignore')

In [15]:
x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

In [16]:
x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)

In [17]:
x_train_rem = x_train.drop(['Sex','Age','Embarked'],axis=1)
x_test_rem = x_test.drop(['Sex','Age','Embarked'],axis=1)

In [18]:
x_train_final = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_final = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [19]:
x_test_final.shape

(179, 8)

# Model Create

In [20]:
model = RandomForestClassifier()

model.fit(x_train_final,y_train)

RandomForestClassifier()

In [21]:
y_pred = model.predict(x_test_final)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
print(accuracy_score(y_test,y_pred))

0.8212290502793296


In [24]:
import pickle

In [25]:
pickle.dump(si_age,open('models/si_age.pkl','wb'))
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_sex,open('models/ohe_embarked.pkl','wb'))
pickle.dump(model,open('models/model.pkl','wb'))